In [1]:
import matplotlib as matplot
import seaborn
import bokeh
import keras
import scipy
import plotly
import plotly.express as pe
import numpy
import pandas as pd
from get_tree_data import ncs1_data
import re
# import block

In [2]:
ncs1 = ncs1_data()
  # ncs1 holds all of the data from NCS 006693
  # Includes ncs1.dxdm (DS0002 DataFrame) and ncs1.survey (DS001 DataFrame)
  # Key Functions:
    # ncs1.search_for_description looks for a descriotion of a column name in ncs1.dxdm or ncs1.survey
  # Key Variables:
    # ncs1.dxdm (DS0002)
    # ncs1.survey (DS0001)
    # ncs1.tree (Tree including descriptions of survey and dxdm columns)
    # ncs1.root (root used in traversing ncs1.tree)

In [3]:
ncs1.get_value_from_string(ncs1.root.iloc[33,0])

,VarName,Description,Root_DF,Start,End,DataFrame,recursion_flag
0,AGO1M1,1MTH AGORAPHOBIA W/ OR W/O PD,DXDM_AGORAPHOBIA,6,6,dxdm,0
1,AGO1M2,1MTH AGORAPHOBIA W/O PD,DXDM_AGORAPHOBIA,7,7,dxdm,0
2,AGO6M1,6MTH AGORAPHOBIA W/ OR W/O PD,DXDM_AGORAPHOBIA,8,8,dxdm,0
3,AGO6M2,6MTH AGORAPHOBIA W/O PD,DXDM_AGORAPHOBIA,9,9,dxdm,0
4,AGOLT1,AGORAPHOBIA W/ OR W/O PD,DXDM_AGORAPHOBIA,10,10,dxdm,0
5,AGOLT2,AGORAPHOBIA W/O PD,DXDM_AGORAPHOBIA,11,11,dxdm,0
6,AGONS,AGO WITH OR W-O PANIC ONSET,DXDM_AGORAPHOBIA,12,12,dxdm,0
7,AGONSA,AGO WITH OR W-O PANIC ONSET AGE,DXDM_AGORAPHOBIA,13,13,dxdm,0
8,AGOYR1,12MTH AGORAPHOBIA W/ OR W/O PD,DXDM_AGORAPHOBIA,14,14,dxdm,0
9,AGOYR2,12MTH AGORAPHOBIA W/O PD,DXDM_AGORAPHOBIA,15,15,dxdm,0


In [9]:
ncs1.get_variable_info('AGREC')

{'varName': 'AGREC',
 'type': 'numeric',
 'missingValues': None,
 'integer': True,
 'formatWidth': 1,
 'decimals': 0,
 'varLabel': 'AGO WITH OR W-O PANIC RECENCY',
 'filter': None,
 'qText': None,
 'txt': None,
 'categories': [{'valueCode': None,
   'valueString': '1',
   'valueLabel': '<= 1 Month',
   'valueTxt': None,
   'missing': False,
   'sysMiss': False,
   'stats': {'freq': '136'}},
  {'valueCode': None,
   'valueString': '2',
   'valueLabel': '>1 Month & <=6 Months',
   'valueTxt': None,
   'missing': False,
   'sysMiss': False,
   'stats': {'freq': '81'}},
  {'valueCode': None,
   'valueString': '3',
   'valueLabel': '>6 Months & <= 1 Year',
   'valueTxt': None,
   'missing': False,
   'sysMiss': False,
   'stats': {'freq': '49'}},
  {'valueCode': None,
   'valueString': '4',
   'valueLabel': '> 1 Year',
   'valueTxt': None,
   'missing': False,
   'sysMiss': False,
   'stats': {'freq': '221'}},
  {'valueCode': None,
   'valueString': 'Sysmiss',
   'valueLabel': None,
   'val

In [27]:
ncs1.dxdm.loc[:,'AGO1M1']

0       0
1       0
2       0
3       0
4       0
       ..
8093    0
8094    0
8095    0
8096    0
8097    0
Name: AGO1M1, Length: 8098, dtype: int64